In [10]:
from sklearn.model_selection import GridSearchCV, KFold, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.preprocessing import StandardScaler
from xgboost import plot_importance
import matplotlib.pyplot as plt
import xgboost as xgb
import pandas as pd
import numpy as np

pd.set_option("display.max_columns", None)

In [11]:
identifier_name = 'flight_id'

features_names = [
    'wtc', 
    'flown_distance', 
    'aircraft_type', 
    'groundspeed_max', 
    'groundspeed_75percentile', 
    'altitude_25percentile', 
    'airline', 
    'longitude_max', 
    'latitude_min', 
    'flight_duration', 
    'vertical_rate_std', 
    'altitude_median', 
    'longitude_mean', 
    'altitude_75percentile', 
    'longitude_std', 
    'latitude_max', 
    'latitude_std', 
    'vertical_rate_max', 
    'vertical_rate_25percentile', 
    'latitude_mean', 
    'longitude_min', 
    'longitude_25percentile', 
    'ades', 
    'adep', 
    'country_code_adep', 
    'vertical_rate_75percentile', 
    'latitude_median', 
    'altitude_mean', 
    'latitude_25percentile', 
    'country_code_ades', 
    'month_day', 
    'actual_offblock_time_hour', 
    'groundspeed_min', 
    'longitude_75percentile', 
    'vertical_rate_median'
]

target_name = 'tow'

global_random_state = 123

In [12]:
encoded_challenge_set = pd.read_csv('data/encoded_challenge_set.csv')
challenge_features = encoded_challenge_set[features_names]
challenge_target = encoded_challenge_set[target_name]

In [13]:
# Function to score model using Root Mean Square Error
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))
    
# Create a scorer
rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Make cross validation strategy
cv_strategy = KFold(n_splits=5, shuffle=True, random_state=global_random_state)

In [14]:
accurate_challenge_data = pd.DataFrame({})

kfold_num = 0
for train_index, test_index in cv_strategy.split(challenge_features):
    kfold_num += 1
    print("k-fold number:", kfold_num)
    X_train, X_test = challenge_features.iloc[train_index], challenge_features.iloc[test_index]
    y_train, y_test = challenge_target.iloc[train_index], challenge_target.iloc[test_index]

    model = xgb.XGBRegressor(
        objective='reg:squarederror', 
        eval_metric='rmse',
        random_state=global_random_state,
        n_estimators=3000,
        colsample_bytree=1.0,
        learning_rate=0.1, 
        max_depth=10, 
        subsample=1.0
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    diff_df = X_test.copy()
    diff_df['tow'] = y_test
    diff_df['y_pred'] = y_pred
    diff_df['difference'] = (diff_df['y_pred'] - diff_df['tow']).abs()
    diff_df['percentage_difference'] = (diff_df['difference'] / diff_df['tow']) * 100

    max_percentage_difference = np.ceil(diff_df['percentage_difference'].max()) 

    for per_diff in range(0, int(max_percentage_difference), 1):
        
        per_diff_df = diff_df[(diff_df['percentage_difference'] <= per_diff+1) & (diff_df['percentage_difference'] >= per_diff)]
        
        if len(per_diff_df) > 0:
            per_diff_df = per_diff_df.drop(columns=['y_pred', 'difference', 'percentage_difference'])
            
            duplicated_dfs = []
            for duplicate in range(per_diff+1):
                duplicated_dfs.append(per_diff_df)
            duplicated_df = pd.concat(duplicated_dfs, ignore_index=True)
            
            accurate_challenge_data = pd.concat([accurate_challenge_data, duplicated_df], ignore_index=True)

k-fold number:  1
k-fold number:  2
k-fold number:  3
k-fold number:  4
k-fold number:  5


In [15]:
display(accurate_challenge_data)

,wtc,flown_distance,aircraft_type,groundspeed_max,groundspeed_75percentile,altitude_25percentile,airline,longitude_max,latitude_min,flight_duration,vertical_rate_std,altitude_median,longitude_mean,altitude_75percentile,longitude_std,latitude_max,latitude_std,vertical_rate_max,vertical_rate_25percentile,latitude_mean,longitude_min,longitude_25percentile,ades,adep,country_code_adep,vertical_rate_75percentile,latitude_median,altitude_mean,latitude_25percentile,country_code_ades,month_day,actual_offblock_time_hour,groundspeed_min,longitude_75percentile,vertical_rate_median,tow
0,0,1524,2,464.0,452.0,27837.50,14,28.725253,41.332306,272,720.733367,27975.0,15.610337,27975.0,8.689943,50.074951,2.718672,4288.0,-64.0,47.542739,2.055148,7.415446,226,385,96,0.0,48.983688,24053.646515,45.797241,26,242,11,142.0,23.403425,0.0,83561.0
1,0,1524,2,464.0,452.0,27837.50,14,28.725253,41.332306,272,720.733367,27975.0,15.610337,27975.0,8.689943,50.074951,2.718672,4288.0,-64.0,47.542739,2.055148,7.415446,226,385,96,0.0,48.983688,24053.646515,45.797241,26,242,11,142.0,23.403425,0.0,83561.0
2,0,1524,2,464.0,452.0,27837.50,14,28.725253,41.332306,272,720.733367,27975.0,15.610337,27975.0,8.689943,50.074951,2.718672,4288.0,-64.0,47.542739,2.055148,7.415446,226,385,96,0.0,48.983688,24053.646515,45.797241,26,242,11,142.0,23.403425,0.0,83561.0
3,0,1524,2,464.0,452.0,27837.50,14,28.725253,41.332306,272,720.733367,27975.0,15.610337,27975.0,8.689943,50.074951,2.718672,4288.0,-64.0,47.542739,2.055148,7.415446,226,385,96,0.0,48.983688,24053.646515,45.797241,26,242,11,142.0,23.403425,0.0,83561.0
4,0,1524,2,464.0,452.0,27837.50,14,28.725253,41.332306,272,720.733367,27975.0,15.610337,27975.0,8.689943,50.074951,2.718672,4288.0,-64.0,47.542739,2.055148,7.415446,226,385,96,0.0,48.983688,24053.646515,45.797241,26,242,11,142.0,23.403425,0.0,83561.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
391,1,735,13,455.0,450.0,20843.75,18,22.362448,50.105026,109,1202.443819,38975.0,17.112983,39000.0,3.460967,60.182361,3.227393,4096.0,-768.0,54.698700,11.092072,14.129728,108,108,74,64.0,54.621094,29591.091954,51.668753,60,162,7,142.0,20.142458,0.0,39788.0
392,1,1489,4,448.0,412.0,29975.00,18,18.268275,40.394028,234,794.267479,30000.0,6.010203,30000.0,6.739345,59.805496,6.024300,4736.0,0.0,51.462263,-3.555263,0.253489,123,222,33,0.0,52.096377,27455.521504,46.316574,67,69,11,93.0,11.665177,0.0,50791.0
393,1,735,13,455.0,450.0,20843.75,18,22.362448,50.105026,109,1202.443819,38975.0,17.112983,39000.0,3.460967,60.182361,3.227393,4096.0,-768.0,54.698700,11.092072,14.129728,108,108,74,64.0,54.621094,29591.091954,51.668753,60,162,7,142.0,20.142458,0.0,39788.0
394,1,1489,4,448.0,412.0,29975.00,18,18.268275,40.394028,234,794.267479,30000.0,6.010203,30000.0,6.739345,59.805496,6.024300,4736.0,0.0,51.462263,-3.555263,0.253489,123,222,33,0.0,52.096377,27455.521504,46.316574,67,69,11,93.0,11.665177,0.0,50791.0


In [16]:
accurate_challenge_features = accurate_challenge_data[features_names]
accurate_challenge_target = accurate_challenge_data[target_name]

In [17]:
rmse_scores = []
for train_index, test_index in cv_strategy.split(accurate_challenge_features):
    X_train, X_test = accurate_challenge_features.iloc[train_index], accurate_challenge_features.iloc[test_index]
    y_train, y_test = accurate_challenge_target.iloc[train_index], accurate_challenge_target.iloc[test_index]

    model = xgb.XGBRegressor(
        objective='reg:squarederror', 
        eval_metric='rmse',
        random_state=global_random_state,
        n_estimators=3000,
        colsample_bytree=1.0,
        learning_rate=0.1, 
        max_depth=10, 
        subsample=1.0
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    rmse_score = rmse(y_test, y_pred)
    rmse_scores.append(rmse_score)

print(f"RMSE Score: ", np.mean(rmse_scores))

RMSE Score:  0.03923630388555407


In [18]:
model = xgb.XGBRegressor(
    objective='reg:squarederror', 
    eval_metric='rmse',
    random_state=global_random_state,
    n_estimators=3000,
    colsample_bytree=1.0,
    learning_rate=0.1, 
    max_depth=10, 
    subsample=1.0
)

model.fit(accurate_challenge_features, accurate_challenge_target)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=1.0, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric='rmse', feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.1, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=10, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=3000, n_jobs=None,
             num_parallel_tree=None, random_state=123, ...)

In [19]:
encoded_submission_set = pd.read_csv('data/encoded_submission_set.csv')
submission_features = encoded_submission_set[features_names]

In [20]:
y_pred = model.predict(submission_features)

In [21]:
submission_df = submission_features.copy()
submission_df[identifier_name] = encoded_submission_set[identifier_name]
submission_df[target_name] = y_pred

In [22]:
display(submission_df[[identifier_name, target_name]])

,flight_id,tow
0,248753821,72693.976562
1,248753822,83560.960938
2,248754498,83560.960938
3,248757623,72694.031250
4,248763603,72694.031250
...,...,...
105954,258066302,72694.031250
105955,258068609,83560.960938
105956,258068876,72694.031250
105957,258064675,72694.031250


In [23]:
submission_df[[identifier_name, target_name]].to_csv('./submissions/my_submission_v17.csv', index=False)